In [4]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import ADASYN

df = pd.read_csv("Base.csv")
df. describe()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,...,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.0,1000000.000000
mean,0.011029,0.562696,0.493694,16.718568,86.587867,33.689080,1.025705e+00,8.661499,1572.692049,5665.296605,...,0.889676,10.839303,0.222988,515.851010,0.025242,7.544940,0.576947,1.018312,0.0,3.288674
std,0.104438,0.290343,0.289125,44.046230,88.406599,12.025799,5.381835e+00,20.236155,1005.374565,3009.380665,...,0.313293,12.116875,0.416251,487.559902,0.156859,8.033106,0.494044,0.180761,0.0,2.209994
min,0.000000,0.100000,0.000001,-1.000000,-1.000000,10.000000,4.036860e-09,-15.530555,1.000000,-170.603072,...,0.000000,-1.000000,0.000000,190.000000,0.000000,-1.000000,0.000000,-1.000000,0.0,0.000000
25%,0.000000,0.300000,0.225216,-1.000000,19.000000,20.000000,7.193246e-03,-1.181488,894.000000,3436.365848,...,1.000000,-1.000000,0.000000,200.000000,0.000000,3.103053,0.000000,1.000000,0.0,1.000000
50%,0.000000,0.600000,0.492153,-1.000000,52.000000,30.000000,1.517574e-02,-0.830507,1263.000000,5319.769349,...,1.000000,5.000000,0.000000,200.000000,0.000000,5.114321,1.000000,1.000000,0.0,3.000000
75%,0.000000,0.800000,0.755567,12.000000,130.000000,40.000000,2.633069e-02,4.984176,1944.000000,7680.717827,...,1.000000,25.000000,0.000000,500.000000,0.000000,8.866131,1.000000,1.000000,0.0,5.000000
max,1.000000,0.900000,0.999999,383.000000,428.000000,90.000000,7.845690e+01,112.956928,6700.000000,16715.565404,...,1.000000,32.000000,1.000000,2100.000000,1.000000,85.899143,1.000000,2.000000,0.0,7.000000


In [6]:
# Convertir a categóricas
df['payment_type'] = df.payment_type.astype('category')
df['employment_status'] = df.employment_status.astype('category')
df['housing_status'] = df.housing_status.astype('category')
df['source'] = df.source.astype('category')
df['device_os'] = df.device_os.astype('category')

In [8]:
# One-hot encoding
df = pd.get_dummies(df, drop_first=True)

In [10]:
# Separar X e y
X = df.drop('fraud_bool', axis=1)
y = df['fraud_bool']

In [12]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [14]:
# Aplicar ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

In [18]:
#Entrenamiento
cat = CatBoostClassifier(verbose=0, random_state=42)
cat.fit(X_train_adasyn, y_train_adasyn)

In [20]:
#Predicción
y_pred_cat = cat.predict(X_test)

In [22]:
print("=== CATBOOST ===")
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_cat))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_cat))

=== CATBOOST ===
Matriz de Confusión:
[[296125    566]
 [  3061    248]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    296691
           1       0.30      0.07      0.12      3309

    accuracy                           0.99    300000
   macro avg       0.65      0.54      0.56    300000
weighted avg       0.98      0.99      0.98    300000



In [ ]:
✅ Muy alto acierto en no fraudes (Clase 0): 99% precisión, 100% recall.

⚠️ Fraudes detectados (Clase 1): solo el 7% de recall. A pesar de que la precisión subió al 30%, no hay gran mejora práctica porque se están escapando el 93% de los fraudes.

📉 El modelo es conservador, evitando falsos positivos a costa de dejar pasar muchos fraudes.

🔍 Parecido al resultado con SMOTE simple, así que parece que CatBoost no se beneficia mucho del uso de ADASYN en este caso.